# Estimate differences in modeled and remotely-sensed SMB

1. Monthly snowline altitudes (SLAs)
2. Equilibrium line altitudes (ELAs)
3. Modeled surface mass balance (SMB) at the remotely-sensed snowline
4. Melt factors of snow ($f_{snow}$)

In [ ]:
import os
from scipy import optimize
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import median_abs_deviation as MAD
from tqdm.auto import tqdm
import glob
import geopandas as gpd

In [ ]:
# Paths for inputs and outputs
scm_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/'
model_dir = os.path.join(scm_dir, 'Rounce_et_al_2023')
# Load glacier boundaries for RGI IDs
aois_fn = os.path.join(scm_dir, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)

## 1. Monthly snowline altitudes

### Modeled SLAs

In [ ]:
# Functions for linearly extrapolating the ELA when modeled SMB < 0 everywhere
def linear_fit(x, m, b):
    return m*x + b
    
def extrapolate_ela_linear(X,y, Iend=8):
    # optimize the linear fit
    p, e = optimize.curve_fit(linear_fit, X[0:Iend+1], y[0:Iend+1])
    # extrapolate where y=0
    ela = linear_fit(0, *p)
    return ela

# Check if file already exists
slas_mod_fn = os.path.join(scm_dir, 'analysis', 'monthly_SLAs_modeled.csv')
if not os.path.exists(slas_mod_fn):
    # load binned model data
    bin_fns = sorted(glob.glob(os.path.join(model_dir, 'glac_SMB_binned', '*.nc')))
    
    # remove binned file names for sites without snow cover observations
    aoi_ids = [x[7:] for x in sorted(aois['RGIId'].drop_duplicates().values)]
    bin_fns = [x for x in bin_fns if os.path.basename(x)[0:7] in aoi_ids]

    # initialize dataframe for results
    slas_mod = pd.DataFrame()

    # iterate over binned file names
    i=0
    for bin_fn in tqdm(bin_fns):
        # open binned data
        bin = xr.open_dataset(bin_fn)
        rgi_id = bin.RGIId.data[0] # grab RGI ID

        # grab data variables
        h = bin.bin_surface_h_initial.data[0] # surface elevation [m]
        b_sum = np.zeros((len(bin.time.data), len(h))) # cumulative SMB
        times = [np.datetime64(x) for x in bin.time.data] # datetimes
        months = list(pd.DatetimeIndex(times).month) # months of each datetime
        slas = np.zeros(len(times)) # initialize SLAs

        # iterate over each time period
        for j, time in enumerate(times):
            # subset binned data to time
            bin_time = bin.isel(time=j)
            # grab the SMB 
            b_sum[j,:] = bin_time.bin_massbalclim_monthly.data[0]
            # add the previous SMB (restart the count in October)
            if months[j] != 10: 
                b_sum[j,:] += b_sum[j-1,:]
            # If all SMB > 0, ELA = minimum elevation
            if all(b_sum[j,:] > 0):
                slas[j] = np.min(h)
            # If SMB is > 0 and < 0 in some places, linearly interpolate ELA
            elif any(b_sum[j,:] < 0) & any(b_sum[j,:] > 0):
                slas[j] = np.interp(0, np.flip(b_sum[j,:]), np.flip(h))
            # If SMB < 0 everywhere, fit a piecewise linear fit and extrapolate for SMB=0
            elif all(b_sum[j,:] < 0):
                X, y = b_sum[j,:], h
                slas[j] = extrapolate_ela_linear(X, y, Iend=5)
            else:
                print('issue')

        # compile in dataframe
        df = pd.DataFrame({'Date': times,
                           'SLA_mod_m': slas})
        
        # Because each SMB value represents the total SMB for each month, add 1 month to the dates
        df['Date'] = df['Date'] + pd.DateOffset(months=1)
        df['RGIId'] = rgi_id

        slas_mod = pd.concat([slas_mod, df])
            
        i+=1

    # Rearrange columns
    slas_mod = slas_mod[['RGIId', 'Date', 'SLA_mod_m']]
    # Save to file
    slas_mod.to_csv(slas_mod_fn, index=False)
    print('Modeled monthly SLAs saved to file:', slas_mod_fn)

else:
    
    slas_mod = pd.read_csv(slas_mod_fn)
    slas_mod['Date'] = pd.DatetimeIndex(slas_mod['Date'])
    print('Modeled monthly SLAs loaded from file.')

slas_mod.reset_index(drop=True, inplace=True)
slas_mod

### Remotely-sensed SLAs

In [ ]:
slas_obs_fn = os.path.join(scm_dir, 'analysis', 'monthly_SLAs_observed.csv')
if not os.path.exists(slas_obs_fn):
    # iterate over RGI IDs
    slas_obs = pd.DataFrame()
    for rgi_id in tqdm(sorted(aois['RGIId'].drop_duplicates().values)):
        scs_fn = os.path.join(scm_dir, 'study-sites', rgi_id, f'{rgi_id}_snow_cover_stats.csv')
        scs = pd.read_csv(scs_fn)
        scs['datetime'] = pd.to_datetime(scs['datetime'], format='mixed')
        scs['Year'] = scs['datetime'].dt.year
        scs['Month'] = scs['datetime'].dt.month
        scs['Day'] = scs['datetime'].dt.day
        # Filter data to within one week of the first of each month
        scs_filtered = scs[(scs['Day'] >= 25) | (scs['Day'] <= 7)]
        # Grab monthly snowline
        Imonths = []
        dates = []
        for year, month in scs_filtered[['Year', 'Month']].drop_duplicates().values:
            first_of_month = pd.Timestamp(year=year, month=month, day=1)
            # identify closest observation to this date
            scs_filtered.loc[:, 'diff'] = np.abs(scs_filtered.loc[:, 'datetime'] - first_of_month)
            Imonths.append(scs_filtered['diff'].idxmin())
            # save date 
            dates.append(pd.Timestamp(f"{year}-{month}-01"))
        scs_monthly = scs.iloc[Imonths].reset_index(drop=True)

        # add date column that is first of month
        scs_monthly['Date'] = dates

        # concatenate to full dataframe
        slas_obs = pd.concat([slas_obs, scs_monthly])

    # select relevant columns
    slas_obs.rename(columns={'ELA_from_AAR_m': 'SLA_obs_m'}, inplace=True)
    slas_obs = slas_obs[['RGIId', 'Date', 'SLA_obs_m']]
    slas_obs.reset_index(drop=True, inplace=True)

    # save to file
    slas_obs.to_csv(slas_obs_fn, index=False)
    print('Remotely-sensed monthly SLAs saved to file:', slas_obs_fn)

else:  
    slas_obs = pd.read_csv(slas_obs_fn)
    slas_obs['Date'] = pd.to_datetime(slas_obs['Date'])
    print('Remotely-sensed monthly SLAs loaded from file.')

slas_obs


### Merge

In [ ]:
### Grab minimum glacier elevations for standardizing ###
# Define output file name
min_sla_obs_fn = os.path.join(scm_dir, 'analysis', 'minimum_glacier_elevations_observed.csv')
if not os.path.exists(min_sla_obs_fn):
    # load AOIs
    aois = gpd.read_file(aois_fn)
    min_slas_obs = pd.DataFrame()
    # Iterate over sites
    for rgi_id in tqdm(aois['RGIId'].drop_duplicates().values):
        # Load snowlines
        scs_fn = os.path.join(scm_dir, 'study-sites', rgi_id, f'{rgi_id}_snow_cover_stats.csv')
        scs = pd.read_csv(scs_fn)
        # Remove any wonky values
        scs.loc[np.abs(scs['ELA_from_AAR_m']) > 1e10] = np.nan
        # Get minimum snowline altitude
        min_sla = scs['ELA_from_AAR_m'].min()
        # Add to dataframe
        df = pd.DataFrame({'RGIId': [rgi_id], 'SLA_obs_m_min': [min_sla]})
        min_slas_obs = pd.concat([min_slas_obs, df], axis=0)

    # Save to file
    min_slas_obs.reset_index(drop=True, inplace=True)
    min_slas_obs.to_csv(min_sla_obs_fn, index=False)
    print('Minimum remotely-sensed glacier elevations saved to file:', min_sla_obs_fn)
    
else:
    min_slas_obs = pd.read_csv(min_sla_obs_fn)
    print('Minimum remotely-sensed glacier elevations loaded.')

min_slas_obs

In [ ]:
# Define output file
slas_merged_fn = os.path.join(scm_dir, 'analysis', 'monthly_SLAs_modeled_observed_merged.csv')
if not os.path.exists(slas_merged_fn):

    # Merge modeled and remotely-sensed ELAs
    slas_merged = slas_mod[['RGIId', 'Date', 'SLA_mod_m']].merge(slas_obs[['RGIId', 'Date', 'SLA_obs_m']],
                                                                 on=['RGIId', 'Date'])
    # Remove 2023 values (no modeled data in 2023)
    slas_merged = slas_merged.loc[pd.DatetimeIndex(slas_merged['Date']).year < 2023]
    
    # Remove observations outside May - September
    slas_merged = slas_merged.loc[(pd.DatetimeIndex(slas_merged['Date']).month >=5) 
                                  & (pd.DatetimeIndex(slas_merged['Date']).month <=9)]

    # Subtract the minimum snowline altitudes to mitigate datum issues, s.t. SLAs are w.r.t. 0 m. 
    for rgi_id in slas_merged['RGIId'].drop_duplicates().values:
        min_sla_obs = min_slas_obs.loc[min_slas_obs['RGIId']==rgi_id, 'SLA_obs_m_min'].values[0]
        slas_merged.loc[slas_merged['RGIId']==rgi_id, 'SLA_obs_m'] -= min_sla_obs
        min_sla_mod = slas_mod.loc[slas_mod['RGIId']==rgi_id, 'SLA_mod_m'].min()
        slas_merged.loc[slas_merged['RGIId']==rgi_id, 'SLA_mod_m'] -= min_sla_mod

    # Save results
    slas_merged.to_csv(slas_merged_fn, index=False)
    print('Merged monthly SLAs saved to file:', slas_merged_fn)

else:
    slas_merged = pd.read_csv(slas_merged_fn)
    print('Merged monthly SLAs loaded.')


slas_merged['SLA_mod-obs_m'] = slas_merged['SLA_mod_m'] - slas_merged['SLA_obs_m']

# Plot
fig, ax = plt.subplots(figsize=(6,5))
ax.hist(slas_merged['SLA_mod-obs_m'], bins=50)
ax.set_xlabel('SLA$_{mod}$ - SLA$_{obs}$ [m]')
ax.set_ylabel('Counts')
plt.show()


print('\nDifference stats:')
print(f'Mean diff = {np.nanmean((slas_merged["SLA_mod-obs_m"]).values)} m')
print(f'Std. diff = {np.nanstd((slas_merged["SLA_mod-obs_m"]).values)} m')
print(f'Median diff = {np.nanmedian((slas_merged["SLA_mod-obs_m"]).values)} m')
print(f'MAD diff = {MAD((slas_merged["SLA_mod-obs_m"]).values, nan_policy="omit")} m')

## 2. ELAs

### Modeled ELAs

In [ ]:
elas_mod_fn = os.path.join(scm_dir, 'analysis', 'annual_ELAs_modeled.csv')
if not os.path.exists(elas_mod_fn):
    # Add Year column
    slas_mod['Year'] = pd.DatetimeIndex(slas_mod['Date']).year
    # Identify the row of maximum ELA for each site and each year
    Imax = slas_mod.groupby(by=['RGIId', 'Year'])['SLA_mod_m'].idxmax().values
    elas_mod = slas_mod.iloc[Imax].reset_index(drop=True)
    elas_mod.rename(columns={'SLA_mod_m': 'ELA_mod_m'}, inplace=True)
    # Reorder columns
    elas_mod = elas_mod[['RGIId', 'Date', 'Year', 'ELA_mod_m']]
    # Save to file
    elas_mod.to_csv(elas_mod_fn, index=False)
    print('Modeled annual ELAs saved to file:', elas_mod_fn)
else:
    elas_mod = pd.read_csv(elas_mod_fn)
    elas_mod['Date'] = pd.to_datetime(elas_mod['Date'])
    print('Modeled annual ELAs loaded from file.')
    
elas_mod

### Remotely-sensed ELAs

In [ ]:
elas_obs_fn = os.path.join(scm_dir, 'analysis', 'annual_ELAs_observed.csv')
if not os.path.exists(elas_obs_fn):
    # iterate over sites
    elas_obs = pd.DataFrame()
    for rgi_id in tqdm(slas_obs['RGIId'].drop_duplicates().values):
        # Subset to site
        slas_obs_site = slas_obs.loc[slas_obs['RGIId']==rgi_id].reset_index(drop=True)
        # Subset to 2016–2023
        slas_obs_site = slas_obs_site.loc[slas_obs_site['Date'].dt.year >= 2016].reset_index(drop=True)
        # identify maximum annual SLA
        imax = slas_obs_site.groupby(slas_obs_site['Date'].dt.year)['SLA_obs_m'].idxmax().values
        df = slas_obs_site.iloc[imax]
        # concatenate to full dataframe
        elas_obs = pd.concat([elas_obs, df])
    elas_obs.reset_index(drop=True, inplace=True)
    elas_obs.rename(columns={'SLA_obs_m': 'ELA_obs_m'}, inplace=True)

    # save to file
    elas_obs.to_csv(elas_obs_fn, index=False)
    print('Remotely-sensed ELAs saved to file:', elas_obs_fn)

else:
    elas_obs = pd.read_csv(elas_obs_fn)
    elas_obs['Date'] = pd.to_datetime(elas_obs['Date'])
    print('Remotely-sensed ELAs loaded from file.')

elas_obs

### Merged

In [ ]:
# Define output file name
elas_merged_fn = os.path.join(scm_dir, 'analysis', 'annual_ELAs_modeled_observed_merged.csv')
if not os.path.exists(elas_merged_fn):

    # Merge modeled and remotely-sensed modeled ELAs
    elas_obs['Year'] = elas_obs['Date'].dt.year
    elas_merged = elas_obs[['RGIId', 'Year', 'ELA_obs_m']].merge(elas_mod[['RGIId', 'Year', 'ELA_mod_m']],
                                                                 on=['RGIId', 'Year'])
    
    # Subset to 2016–2022 (no modeled data in 2023)
    elas_merged = elas_merged.loc[(elas_merged['Year'] >= 2016) 
                                                & (elas_merged['Year'] < 2023)]
        
    # Save results
    elas_merged.to_csv(elas_merged_fn, index=False)
    print('Merged annual ELAs saved to file:', elas_merged_fn)

else:
    elas_merged = pd.read_csv(elas_merged_fn)
    print('Merged annual ELAs loaded.')
    
# Calculate difference
elas_merged['ELA_mod-obs_m'] = elas_merged['ELA_mod_m'] - elas_merged['ELA_obs_m']

# Plot
fig, ax = plt.subplots(figsize=(5,5))
ax.hist(elas_merged['ELA_mod-obs_m'], bins=50)
ax.set_xlabel('ELA$_{mod}$ - ELA$_{obs}$ [m]')
ax.set_ylabel('Counts')
plt.show()

print('\nDifference stats:')
print(f"Mean diff = {np.nanmean(elas_merged['ELA_mod-obs_m'])} m")
print(f"Std. diff = {np.nanstd(elas_merged['ELA_mod-obs_m'])} m")
print(f"Median diff = {np.nanmedian(elas_merged['ELA_mod-obs_m'])} m")
print(f"MAD diff = {MAD(elas_merged['ELA_mod-obs_m'], nan_policy='omit')} m")

## 3. Modeled SMB at remotely-sensed snowlines

In [ ]:
sla_obs_smb_mod_fn = os.path.join(scm_dir, 'analysis', 'modeled_SMB_at_observed_SLA.csv')
if not os.path.exists(sla_obs_smb_mod_fn):
    # Intialize dataframe
    sla_obs_smb_mod = pd.DataFrame()
    
    # Iterate over RGI IDs
    for rgi_id in tqdm(aois['RGIId'].drop_duplicates().values):
        mod_smb_fn = glob.glob(os.path.join(model_dir, 'glac_SMB_binned', f"{rgi_id.split('RGI60-0')[1]}*.nc"))[0]
        mod_smb = xr.open_dataset(mod_smb_fn)
        slas_obs_site = slas_obs.loc[slas_obs['RGIId']==rgi_id]

        # grab data variables
        h = mod_smb.bin_surface_h_initial.data[0] # surface elevation [m]
        b_sum = np.zeros((len(mod_smb.time.data), len(h))) # cumulative SMB
        dts = [pd.Timestamp(np.datetime64(x)) for x in mod_smb.time.data] # datetimes
        months = list(pd.DatetimeIndex(dts).month) # months of each datetime

        # iterate over each time period
        for j, dt in enumerate(dts):
            # subset binned data to time
            mod_smb_time = mod_smb.isel(time=j)
            # grab the SMB 
            b_sum[j,:] = mod_smb_time.bin_massbalclim_monthly.data[0]
            # add the previous SMB (restart the count in October)
            if months[j] != 10: 
                b_sum[j,:] += b_sum[j-1,:]
            # grab observed SLA
            sla = slas_obs_site.loc[slas_obs_site['Date']==dt, 'SLA_obs_m'].values
            if len(sla) > 0:
                # interpolate SMB at SLA
                smb_sla = np.interp(sla[0], h, b_sum[j,:])
                df = pd.DataFrame({'RGIId': [rgi_id],
                                   'Date': [dt],
                                   'SMB_at_SLA_mwe': [smb_sla]})
                # add to full dataframe
                sla_obs_smb_mod = pd.concat([sla_obs_smb_mod, df], axis=0)
                
    # Save to file
    sla_obs_smb_mod.reset_index(drop=True, inplace=True)
    sla_obs_smb_mod.to_csv(sla_obs_smb_mod_fn, index=False)
    print('SMB at SLA saved to file:', sla_obs_smb_mod_fn)

else:
    sla_obs_smb_mod = pd.read_csv(sla_obs_smb_mod_fn)
    print('SMB at SLA loaded from file.')
    
sla_obs_smb_mod
    
                
            

## 4. Melt factors of snow

### Modeled

In [ ]:
# Check if already exists in file
fsnow_mod_fn = os.path.join(scm_dir, 'analysis', 'fsnow_modeled.csv')
if not os.path.exists(fsnow_mod_fn):
    print('Compiling modeled melt factors of snow')
    # Load AOIs for RGI IDs
    aois_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/analysis/all_aois.shp'
    aois = gpd.read_file(aois_fn)
    modelprm_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/Rounce_et_al_2023/modelprms'
    # Initialize dataframe
    fsnow_mod = pd.DataFrame()
    # Iterate over RGI IDs
    for rgi_id in tqdm(aois['RGIId'].drop_duplicates().values):
        # Load model parameters
        modelprm_fn = os.path.join(modelprm_dir, f"{rgi_id.replace('RGI60-0','')}-modelprms_dict.pkl")
        modelprm = pd.read_pickle(modelprm_fn)
        # Take the median of MCMC fsnow results (not much different than the mean)
        ddfsnow_mcmc = np.array(modelprm['MCMC']['ddfsnow']['chain_0'])
        df = pd.DataFrame({"RGIId": [rgi_id],
                           "fsnow_mod": [np.median(ddfsnow_mcmc)]})
        # Concatenate df to full dataframe
        fsnow_mod = pd.concat([fsnow_mod, df])
    # Save to file
    fsnow_mod.reset_index(drop=True, inplace=True)
    fsnow_mod.to_csv(fsnow_mod_fn, index=False)
    print('Compiled melt factors of snow saved to file:', fsnow_mod_fn)

else:
    fsnow_mod = pd.read_csv(fsnow_mod_fn)
    print('Compiled melt factors of snow loaded from file.')

plt.hist(fsnow_mod['fsnow_mod'], bins=np.linspace(0, 0.006, 50))
plt.show()



### Observed

$Accumulation - Melt = SMB$

$\Sigma(Precip_{solid}) - \Sigma(PDDs)*f_{snow} = SMB$

At the snowline, SMB = 0. Solve for $f_{snow}$ at observed snowline altitudes.

$f_{snow} = \frac{\Sigma Precip_{solid} (h_{sl})}{\Sigma PDDs (h_{sl})}$

In [ ]:
fsnow_obs_fn = os.path.join(scm_dir, 'analysis', 'fsnow_observed.csv')
if not os.path.exists(fsnow_obs_fn):

    # Load calculated lapse rates
    lapse_fn = os.path.join(model_dir, 'ERA5_lapserates_monthly.nc')
    lapse = xr.open_dataset(lapse_fn)

    # Load ERA5 geopotential, convert to geoidal height
    gp_fn = os.path.join(model_dir, 'ERA5_geopotential.nc')
    gp = xr.open_dataset(gp_fn).squeeze()
    gp['h'] = gp['z'] / 9.81
    # gp['h'].plot(vmin=-100, vmax=5e3)

    # Load ERA5 monthly temperatures and precipitation
    temp_fn = os.path.join(model_dir, 'ERA5_temp_monthly.nc')
    temp = xr.open_dataset(temp_fn)
    precip_fn = os.path.join(model_dir, 'ERA5_totalprecip_monthly.nc')
    precip = xr.open_dataset(precip_fn)

    # Iterate over glaciers
    fsnow_obs = pd.DataFrame()
    for rgi_id in tqdm(slas_mod['RGIId'].drop_duplicates().values):

        # Load monthly remotely-sensed SLAs
        slas_obs_glacier = slas_obs.loc[slas_obs['RGIId']==rgi_id]
        slas_obs_glacier = slas_obs_glacier.loc[slas_obs_glacier['Date'] <= temp.time.max().values]

        # Load modeled monthly SMB
        smb_fn = glob.glob(os.path.join(model_dir, 'glac_SMB_binned', f"{rgi_id.split('RGI60-0')[1]}*.nc"))[0]
        smb = xr.open_dataset(smb_fn)

        # Subset model files to glacier
        lat, lon = smb.CenLat.values[0], smb.CenLon.values[0] + 360
        gp_glacier = gp.sel(latitude=lat, longitude=lon, method='nearest')
        lapse_glacier = lapse.sel(latitude=lat, longitude=lon, method='nearest')
        temp_glacier = temp.sel(latitude=lat, longitude=lon, method='nearest')
        precip_glacier = precip.sel(latitude=lat, longitude=lon, method='nearest')
        # average over the "expver" dimension
        temp_glacier = temp_glacier.mean(dim='expver') 
        precip_glacier = precip_glacier.mean(dim='expver') 
        # subset to 2012 on
        temp_glacier = temp_glacier.sel(time=slice("2012-10-01", None))
        precip_glacier = precip_glacier.sel(time=slice("2012-10-01", None))

        # Convert temperatures in K to C
        temp_glacier['t2m_C'] = temp_glacier['t2m'] - 273.15
        
        # # Resample temperatures to daily for PDD calculations
        # temp_glacier = temp_glacier.resample('1D').interpolate("linear")

        # Difference ERA5 heights from glacier elevations
        h = smb.bin_surface_h_initial
        elev_diff = h - gp_glacier['h'] - 2 # account for 2m temperature

        # Apply lapse rates to temperatures
        temp_glacier['t2m_C_adj'] = temp_glacier['t2m_C'] + (lapse_glacier['lapserate'] * elev_diff)

        # Calculate PDDs
        temp_glacier['PDD'] = xr.where(temp_glacier['t2m_C_adj'] > 0, temp_glacier['t2m_C_adj'], 0) # by month
        temp_glacier['PDD'] = temp_glacier['PDD'] * temp_glacier.time.dt.days_in_month
        def water_year_da(time):
            year = time.dt.year
            return xr.where(time.dt.month >= 10, year, year - 1)
        temp_glacier = temp_glacier.assign_coords(water_year=water_year_da(temp_glacier['time']))
        temp_glacier['PDD_cumsum'] = temp_glacier['PDD'].groupby("water_year").cumsum(dim="time")

        # Estimate snow as precipitation when temperatures are positive
        precip_glacier['snow'] = xr.where(temp_glacier['t2m_C_adj'] < 0, precip_glacier['tp'], 0) # per month
        precip_glacier['snow'] = precip_glacier['snow'] * precip_glacier.time.dt.days_in_month # per day

        # Calculate cumulative annual snowfall
        precip_glacier = precip_glacier.assign_coords(water_year=water_year_da(precip_glacier['time']))
        precip_glacier['snow_cumsum'] = precip_glacier['snow'].groupby("water_year").cumsum(dim="time")

        # Estimate melt factors of snow
        fsnow_df = pd.DataFrame()
        dates = slas_obs_glacier['Date'].values
        fsnows = np.zeros(len(dates))
        h_adj = h - h.min() # remove minimum elevation for comparison with observed
        for j, date in enumerate(dates):
            sla_obs_date = slas_obs_glacier.loc[slas_obs_glacier['Date']==date, 'SLA_obs_m'].values[0]
            sla_obs_date -= min_slas_obs.loc[min_slas_obs['RGIId']==rgi_id, 'SLA_obs_m_min'].values[0]
            pdd_sum = np.interp(sla_obs_date, 
                                h.values.ravel(), 
                                temp_glacier.sel(time=date)['PDD_cumsum'].values.ravel())
            snow_sum = np.interp(sla_obs_date, 
                                 h.values.ravel(), 
                                 precip_glacier.sel(time=date)['snow_cumsum'].values.ravel())
            if pdd_sum==0:
                fsnows[j] = 0
            else:
                fsnows[j] = snow_sum / pdd_sum

        df = pd.DataFrame({'RGIId': [rgi_id],
                           'fsnow_obs': [np.nanmedian(fsnows)]})   
        fsnow_obs = pd.concat([fsnow_obs, df], axis=0)
        
    # Save to file
    fsnow_obs.to_csv(fsnow_obs_fn, index=False)
    print('Observed melt factors of snow saved to file:', fsnow_obs_fn)

else:
    fsnow_obs = pd.read_csv(fsnow_obs_fn)
    print('Observed melt factors of snow loaded from file.')
    
plt.hist(fsnow_obs['fsnow_obs'], bins=np.linspace(0, 0.0006, 50))
plt.show()


In [ ]:
precip_glacier.mean(dim='bin')['snow_cumsum'].plot()

In [ ]:
temp_glacier.mean(dim='bin')['PDD_cumsum'].plot()